# Lesson 3 Module 3: Asymptotic Efficiency & Cramér–Rao Lower Bound

This notebook demonstrates Fisher information and the Cramér–Rao lower bound using practical examples.
It extends Lesson 2 (Fisher information, MLE) and provides foundation for confidence intervals.

## Learning Objectives
- Define score function, Fisher information, and CRLB for unbiased estimators
- State regularity conditions and equality cases
- Explain asymptotic normality of MLEs and asymptotic efficiency
- Work through Normal, Poisson, and Exponential examples (Lesson 2)

## Repository Context
- Uses `fisher_info_*` functions from the appendix
- Extends Lesson 2 MLE properties and Fisher information concepts
- Demonstrates efficiency of Lesson 2 estimators

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats

# Set style and random seed
sns.set_theme(context="talk", style="whitegrid")
sns.set_palette(["#000000", "#E69F00", "#56B4E9", "#009E73",
                 "#F0E442", "#0072B2", "#D55E00", "#CC79A7"])
rng = np.random.default_rng(2025)

print("Environment setup complete. Random seed: 2025")

## 1. Fisher Information Functions

Using the functions from the appendix to compute Fisher information for common distributions.

In [ ]:
# Fisher information functions (from appendix)
def fisher_info_normal_mean(sigma):
    """Per-observation Fisher information for μ in Normal(μ, σ²) with σ known."""
    return 1.0 / (sigma**2)

def fisher_info_poisson(lam):
    """Per-observation Fisher information for λ in Poisson(λ)."""
    return 1.0 / lam

def fisher_info_exponential(lam):
    """Per-observation Fisher information for λ in Exponential(rate=λ)."""
    return 1.0 / (lam**2)

print("Fisher information functions defined")

In [ ]:
# Demonstrate Fisher information for different parameters
sigma_values = np.array([0.5, 1.0, 2.0, 5.0])
lambda_values = np.array([0.5, 1.0, 2.0, 5.0])

print("Fisher Information Comparison:")
print("σ\tI_N(μ) = 1/σ²")
for sigma in sigma_values:
    print(f"{sigma}\t{fisher_info_normal_mean(sigma):.3f}")

print("\nλ\tI_P(λ) = 1/λ")
for lam in lambda_values:
    print(f"{lam}\t{fisher_info_poisson(lam):.3f}")

print("\nλ\tI_E(λ) = 1/λ²")
for lam in lambda_values:
    print(f"{lam}\t{fisher_info_exponential(lam):.3f}")

## 2. Normal Mean: CRLB Achievement

Demonstrate that the sample mean achieves the CRLB for Normal data.

In [ ]:
def simulate_crlb_achievement(true_mu, true_sigma, n_values, R=5000):
    """
    Simulate CRLB achievement for Normal mean estimation.
    """
    results = []

    for n in n_values:
        estimates = np.zeros(R)

        for r in range(R):
            sample = rng.normal(true_mu, true_sigma, n)
            estimates[r] = np.mean(sample)

        empirical_var = np.var(estimates, ddof=0)
        crlb = true_sigma**2 / n
        efficiency_ratio = empirical_var / crlb

        results.append({
            'n': n,
            'empirical_var': empirical_var,
            'crlb': crlb,
            'efficiency_ratio': efficiency_ratio
        })

    return pd.DataFrame(results)

print("Function defined: simulate_crlb_achievement")

In [ ]:
# Simulate CRLB achievement
true_mu = 10.0
true_sigma = 2.0
n_values = [5, 10, 20, 50, 100, 200]

normal_results = simulate_crlb_achievement(true_mu, true_sigma, n_values, R=10000)
print("Normal mean CRLB achievement:")
print(normal_results.round(6))

In [ ]:
# Plot CRLB achievement
fig, axes = plt.subplots(1, 2, figsize=(15, 6))

# Variance comparison
axes[0].plot(normal_results['n'], normal_results['empirical_var'], 'b-', linewidth=3, marker='o', label='Empirical Var')
axes[0].plot(normal_results['n'], normal_results['crlb'], 'r-', linewidth=3, marker='s', label='CRLB')
axes[0].set_xlabel('Sample Size')
axes[0].set_ylabel('Variance')
axes[0].set_title('Sample Mean Variance vs CRLB')
axes[0].legend()
axes[0].grid(True, alpha=0.3)

# Efficiency ratio
axes[1].plot(normal_results['n'], normal_results['efficiency_ratio'], 'g-', linewidth=3, marker='D')
axes[1].axhline(1.0, color='red', linestyle='--', alpha=0.7, label='Perfect Efficiency')
axes[1].set_xlabel('Sample Size')
axes[1].set_ylabel('Efficiency Ratio')
axes[1].set_title('Efficiency Ratio (Empirical/CRLB)')
axes[1].legend()
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.savefig('../slides/figures/normal_crlb_achievement.png', dpi=150, bbox_inches='tight')
plt.show()

print(f"Asymptotic efficiency ratio: {normal_results.iloc[-1]['efficiency_ratio']:.4f}")
print("Sample mean achieves CRLB asymptotically")

## 3. Poisson Rate: MLE Efficiency

Demonstrate CRLB achievement for Poisson MLE.

### Fisher Information and CRLB for Selected Rates

We start by verifying the analytic Fisher information $I(\lambda)=1/\lambda$ for a few Poisson rates and summarise the implied single-observation CRLB. A simulation check confirms the variance-of-score definition matches the analytic value.


In [ ]:
lambda_values_focus = np.array([2.0, 5.0, 10.0])

def summarise_poisson_fisher(lambda_vals, n_sims=100_000):
    records = []
    score_cache = {}
    sample_cache = {}

    for lam in lambda_vals:
        analytical = fisher_info_poisson(lam)

        samples = rng.poisson(lam, n_sims)
        scores = samples / lam - 1.0
        numerical = np.var(scores)

        records.append({
            'lambda': lam,
            'I_analytical': analytical,
            'I_numerical': numerical,
            'CRLB_single': 1.0 / analytical
        })

        score_cache[lam] = scores[:2000]
        sample_cache[lam] = samples[:2000]

    summary_df = pd.DataFrame(records)
    return summary_df, score_cache, sample_cache

poisson_info_df, poisson_score_cache, poisson_sample_cache = summarise_poisson_fisher(lambda_values_focus)
poisson_info_df.round({'I_analytical': 6, 'I_numerical': 6, 'CRLB_single': 6})


In [ ]:
fig, axes = plt.subplots(2, 2, figsize=(14, 10))

lambda_grid = np.linspace(0.5, 15, 200)
axes[0, 0].plot(lambda_grid, 1.0 / lambda_grid, linewidth=2, color='steelblue', label=r'$I(\lambda) = 1/\lambda$')
axes[0, 0].scatter(poisson_info_df['lambda'], poisson_info_df['I_analytical'],
                   color='crimson', s=80, zorder=5, label='Selected rates')
axes[0, 0].set_xlabel('Poisson rate $\lambda$')
axes[0, 0].set_ylabel('Fisher information $I(\lambda)$')
axes[0, 0].set_title('Per-observation information')
axes[0, 0].grid(True, alpha=0.3)
axes[0, 0].legend()

axes[0, 1].plot(lambda_grid, lambda_grid, linewidth=2, color='darkorange', label='CRLB, n=1')
axes[0, 1].plot(lambda_grid, lambda_grid / 10.0, linewidth=2, color='seagreen', linestyle='--', label='CRLB, n=10')
axes[0, 1].scatter(poisson_info_df['lambda'], poisson_info_df['CRLB_single'], color='crimson', s=80, zorder=5)
axes[0, 1].set_xlabel('Poisson rate $\lambda$')
axes[0, 1].set_ylabel('Variance lower bound')
axes[0, 1].set_title('Cramér–Rao bounds')
axes[0, 1].grid(True, alpha=0.3)
axes[0, 1].legend()

for lam, scores in poisson_score_cache.items():
    axes[1, 0].hist(scores, bins=40, density=True, alpha=0.5, label=f'λ = {lam:.0f}')
axes[1, 0].axvline(0.0, color='black', linestyle='--', alpha=0.6)
axes[1, 0].set_xlabel('Score $U(\lambda) = X/\lambda - 1$')
axes[1, 0].set_ylabel('Density')
axes[1, 0].set_title('Score distributions (simulated)')
axes[1, 0].legend()
axes[1, 0].grid(True, alpha=0.3)

for lam, samples in poisson_sample_cache.items():
    support = np.arange(0, samples.max() + 1)
    pmf = stats.poisson.pmf(support, lam)
    axes[1, 1].stem(support, pmf, linefmt='-', markerfmt='o', basefmt=' ', label=f'λ = {lam:.0f}', use_line_collection=True)
axes[1, 1].set_xlabel('$x$')
axes[1, 1].set_ylabel('P(X = x)')
axes[1, 1].set_title('Poisson pmfs for selected rates')
axes[1, 1].legend()
axes[1, 1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()


The table confirms $I(\lambda)=1/\lambda$ and the plots visualise how the information and CRLB scale with the rate. The score histograms centre at zero and have variance matching the analytic information, reinforcing the simulation check before we examine the sampling distribution of the Poisson MLE below.


In [ ]:
def simulate_poisson_efficiency(true_lambda, n_values, R=5000):
    """
    Simulate CRLB achievement for Poisson rate estimation.
    """
    results = []

    for n in n_values:
        estimates = np.zeros(R)

        for r in range(R):
            sample = rng.poisson(true_lambda, n)
            estimates[r] = np.mean(sample)  # MLE for Poisson

        empirical_var = np.var(estimates, ddof=0)
        crlb = true_lambda / n
        efficiency_ratio = empirical_var / crlb

        results.append({
            'n': n,
            'empirical_var': empirical_var,
            'crlb': crlb,
            'efficiency_ratio': efficiency_ratio
        })

    return pd.DataFrame(results)

print("Function defined: simulate_poisson_efficiency")

In [ ]:
# Simulate Poisson efficiency
true_lambda = 3.0
poisson_results = simulate_poisson_efficiency(true_lambda, n_values, R=10000)
print("Poisson rate CRLB achievement:")
print(poisson_results.round(6))

In [ ]:
# Plot Poisson CRLB achievement
fig, axes = plt.subplots(1, 2, figsize=(15, 6))

# Variance comparison
axes[0].plot(poisson_results['n'], poisson_results['empirical_var'], 'b-', linewidth=3, marker='o', label='Empirical Var')
axes[0].plot(poisson_results['n'], poisson_results['crlb'], 'r-', linewidth=3, marker='s', label='CRLB')
axes[0].set_xlabel('Sample Size')
axes[0].set_ylabel('Variance')
axes[0].set_title('Poisson MLE Variance vs CRLB')
axes[0].legend()
axes[0].grid(True, alpha=0.3)

# Efficiency ratio
axes[1].plot(poisson_results['n'], poisson_results['efficiency_ratio'], 'g-', linewidth=3, marker='D')
axes[1].axhline(1.0, color='red', linestyle='--', alpha=0.7, label='Perfect Efficiency')
axes[1].set_xlabel('Sample Size')
axes[1].set_ylabel('Efficiency Ratio')
axes[1].set_title('Poisson MLE Efficiency Ratio')
axes[1].legend()
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.savefig('../slides/figures/poisson_crlb_achievement.png', dpi=150, bbox_inches='tight')
plt.show()

print(f"Poisson MLE asymptotic efficiency ratio: {poisson_results.iloc[-1]['efficiency_ratio']:.4f}")

## 4. Exponential Rate: Asymptotic Efficiency

Compare MLE and unbiased estimators for exponential rate.

In [ ]:
def exponential_mle(x):
    """MLE for exponential rate: n / sum(x)"""
    return len(x) / np.sum(x)

def exponential_unbiased(x):
    """Unbiased estimator for exponential rate: (n-1) / sum(x)"""
    n = len(x)
    return (n - 1) / np.sum(x)

def simulate_exponential_efficiency(true_lambda, n_values, R=5000):
    """
    Compare MLE and unbiased estimators for exponential rate.
    """
    results = []

    for n in n_values:
        mle_estimates = np.zeros(R)
        unbiased_estimates = np.zeros(R)

        for r in range(R):
            # Generate exponential with rate lambda (scale = 1/lambda)
            sample = rng.exponential(1/true_lambda, n)
            mle_estimates[r] = exponential_mle(sample)
            unbiased_estimates[r] = exponential_unbiased(sample)

        # MLE properties
        mle_var = np.var(mle_estimates, ddof=0)
        mle_bias = np.mean(mle_estimates) - true_lambda
        mle_mse = np.mean((mle_estimates - true_lambda)**2)

        # Unbiased properties
        unbiased_var = np.var(unbiased_estimates, ddof=0)
        unbiased_bias = np.mean(unbiased_estimates) - true_lambda
        unbiased_mse = np.mean((unbiased_estimates - true_lambda)**2)

        # CRLB
        crlb = true_lambda**2 / n

        results.append({
            'n': n,
            'mle_var': mle_var,
            'mle_bias': mle_bias,
            'mle_mse': mle_mse,
            'unbiased_var': unbiased_var,
            'unbiased_bias': unbiased_bias,
            'unbiased_mse': unbiased_mse,
            'crlb': crlb
        })

    return pd.DataFrame(results)

print("Function defined: simulate_exponential_efficiency")

In [ ]:
# Simulate exponential efficiency
true_lambda = 2.0
exp_results = simulate_exponential_efficiency(true_lambda, n_values, R=10000)
print("Exponential rate estimator comparison:")
print(exp_results.round(6))

In [ ]:
# Plot exponential comparison
fig, axes = plt.subplots(2, 2, figsize=(15, 10))

# Variance comparison
axes[0,0].plot(exp_results['n'], exp_results['mle_var'], 'b-', linewidth=3, marker='o', label='MLE')
axes[0,0].plot(exp_results['n'], exp_results['unbiased_var'], 'r-', linewidth=3, marker='s', label='Unbiased')
axes[0,0].plot(exp_results['n'], exp_results['crlb'], 'g-', linewidth=3, marker='^', label='CRLB')
axes[0,0].set_xlabel('Sample Size')
axes[0,0].set_ylabel('Variance')
axes[0,0].set_title('Exponential Estimator Variance')
axes[0,0].legend()
axes[0,0].grid(True, alpha=0.3)

# Bias comparison
axes[0,1].plot(exp_results['n'], exp_results['mle_bias'], 'b-', linewidth=3, marker='o', label='MLE')
axes[0,1].plot(exp_results['n'], exp_results['unbiased_bias'], 'r-', linewidth=3, marker='s', label='Unbiased')
axes[0,1].axhline(0, color='black', linestyle='--', alpha=0.7)
axes[0,1].set_xlabel('Sample Size')
axes[0,1].set_ylabel('Bias')
axes[0,1].set_title('Exponential Estimator Bias')
axes[0,1].legend()
axes[0,1].grid(True, alpha=0.3)

# MSE comparison
axes[1,0].plot(exp_results['n'], exp_results['mle_mse'], 'b-', linewidth=3, marker='o', label='MLE')
axes[1,0].plot(exp_results['n'], exp_results['unbiased_mse'], 'r-', linewidth=3, marker='s', label='Unbiased')
axes[1,0].set_xlabel('Sample Size')
axes[1,0].set_ylabel('MSE')
axes[1,0].set_title('Exponential Estimator MSE')
axes[1,0].legend()
axes[1,0].grid(True, alpha=0.3)

# Efficiency ratios
mle_efficiency = exp_results['mle_var'] / exp_results['crlb']
unbiased_efficiency = exp_results['unbiased_var'] / exp_results['crlb']

axes[1,1].plot(exp_results['n'], mle_efficiency, 'b-', linewidth=3, marker='o', label='MLE')
axes[1,1].plot(exp_results['n'], unbiased_efficiency, 'r-', linewidth=3, marker='s', label='Unbiased')
axes[1,1].axhline(1.0, color='black', linestyle='--', alpha=0.7, label='Perfect Efficiency')
axes[1,1].set_xlabel('Sample Size')
axes[1,1].set_ylabel('Efficiency Ratio')
axes[1,1].set_title('Asymptotic Efficiency')
axes[1,1].legend()
axes[1,1].grid(True, alpha=0.3)

plt.tight_layout()
plt.savefig('../slides/figures/exponential_efficiency.png', dpi=150, bbox_inches='tight')
plt.show()

print(f"MLE asymptotic efficiency ratio: {mle_efficiency.iloc[-1]:.4f}")
print(f"Unbiased asymptotic efficiency ratio: {unbiased_efficiency.iloc[-1]:.4f}")
print("MLE achieves CRLB asymptotically but has finite-sample bias")


## 5. Confidence Interval Simulations

These scenarios mirror the confidence-interval exercises. Each block reproduces the computations and visualisations directly in the notebook so they can be rerun or adapted.




### Exercise 1: $t$-Interval via the Pivotal Method

We build a reusable helper for the one-sample $t$-interval and illustrate it on synthetic Normal data with unknown variance.



In [ ]:

from dataclasses import dataclass

@dataclass
class TIntervalResult:
    ci_lower: float
    ci_upper: float
    sample_mean: float
    sample_sd: float
    t_crit: float
    n: int

def t_interval(sample, confidence=0.95):
    sample = np.asarray(sample, dtype=float)
    n = sample.size
    xbar = sample.mean()
    s = sample.std(ddof=1)
    alpha = 1.0 - confidence
    t_crit = stats.t.ppf(1.0 - alpha / 2.0, df=n - 1)
    margin = t_crit * s / np.sqrt(n)
    return TIntervalResult(xbar - margin, xbar + margin, xbar, s, t_crit, n)


In [ ]:

sample_data = rng.normal(loc=170.0, scale=10.0, size=20)
t_result = t_interval(sample_data, confidence=0.95)
print(f"Sample size: n = {t_result.n}")
print(f"Sample mean: x¯ = {t_result.sample_mean:.2f}")
print(f"Sample sd: s = {t_result.sample_sd:.2f}")
print(f"t critical value: {t_result.t_crit:.3f}")
print(f"95% CI: [{t_result.ci_lower:.2f}, {t_result.ci_upper:.2f}]")



### Exercise 2: Wilson vs Wald Interval

For $\hat p = 0.5$ we compare Wilson and Wald intervals over increasing sample sizes and observe their convergence.



In [ ]:

def wald_interval(p_hat, n, alpha=0.05):
    z = stats.norm.ppf(1.0 - alpha / 2.0)
    se = np.sqrt(p_hat * (1.0 - p_hat) / n)
    return p_hat - z * se, p_hat + z * se

def wilson_interval(x, n, alpha=0.05):
    z = stats.norm.ppf(1.0 - alpha / 2.0)
    z2 = z**2
    p_tilde = (x + z2 / 2.0) / (n + z2)
    se = np.sqrt(p_tilde * (1.0 - p_tilde) / (n + z2))
    return p_tilde - z * se, p_tilde + z * se

def compare_wald_wilson(sample_sizes, p_hat=0.5, alpha=0.05):
    records = []
    for n in sample_sizes:
        x = int(round(p_hat * n))
        wald_ci = wald_interval(p_hat, n, alpha=alpha)
        wilson_ci = wilson_interval(x, n, alpha=alpha)
        diff = max(abs(wald_ci[0] - wilson_ci[0]), abs(wald_ci[1] - wilson_ci[1]))
        records.append({
            'n': n,
            'wald_lower': wald_ci[0],
            'wald_upper': wald_ci[1],
            'wilson_lower': wilson_ci[0],
            'wilson_upper': wilson_ci[1],
            'max_abs_diff': diff
        })
    return pd.DataFrame(records)

ci_comparison_df = compare_wald_wilson([10, 20, 50, 100, 500, 1000])
ci_comparison_df.round(6)



### Exercise 3: Proportion CIs on A/B Click Data

We compute Wald, Wilson, Agresti–Coull, and Clopper–Pearson intervals for each variant and compare the click-through rates.



In [ ]:
from pathlib import Path

def compute_cis_aggregated(impressions, clicks, confidence=0.95):
    n = impressions
    x = clicks
    p_hat = x / n
    alpha = 1.0 - confidence
    z = stats.norm.ppf(1.0 - alpha / 2.0)

    se_wald = np.sqrt(p_hat * (1.0 - p_hat) / n)
    wald = (max(0.0, p_hat - z * se_wald), min(1.0, p_hat + z * se_wald))

    z2 = z**2
    p_tilde = (x + z2 / 2.0) / (n + z2)
    se_wilson = np.sqrt(p_tilde * (1.0 - p_tilde) / (n + z2))
    wilson = (p_tilde - z * se_wilson, p_tilde + z * se_wilson)

    n_tilde = n + z2
    x_tilde = x + z2 / 2.0
    p_ac = x_tilde / n_tilde
    se_ac = np.sqrt(p_ac * (1.0 - p_ac) / n_tilde)
    agresti_coull = (p_ac - z * se_ac, p_ac + z * se_ac)

    clopper_pearson = (
        stats.beta.ppf(alpha / 2.0, x, n - x + 1) if x > 0 else 0.0,
        stats.beta.ppf(1.0 - alpha / 2.0, x + 1, n - x) if x < n else 1.0
    )

    return {
        'p_hat': p_hat,
        'n': n,
        'x': x,
        'wald': wald,
        'wilson': wilson,
        'agresti_coull': agresti_coull,
        'clopper_pearson': clopper_pearson
    }

ab_path = Path('../../shared/data/ab_test_clicks.csv')
ab_df = pd.read_csv(ab_path)
groups = ab_df.groupby('variant').agg({'impressions': 'sum', 'clicks': 'sum'}).reset_index()

ab_results = {}
for _, row in groups.iterrows():
    ab_results[row['variant']] = compute_cis_aggregated(int(row['impressions']), int(row['clicks']))

display_records = []
for variant, res in ab_results.items():
    for method in ['wald', 'wilson', 'agresti_coull', 'clopper_pearson']:
        lower, upper = res[method]
        display_records.append({
            'variant': variant,
            'method': method.replace('_', ' ').title(),
            'lower': lower,
            'upper': upper,
            'width': upper - lower
        })

proportion_cis_df = pd.DataFrame(display_records)
proportion_cis_df.round({'lower': 4, 'upper': 4, 'width': 4})


In [ ]:

fig, axes = plt.subplots(1, 2, figsize=(14, 6))
method_labels = ['Wald', 'Wilson', 'Agresti-Coull', 'Clopper-Pearson']
colors = ['firebrick', 'royalblue', 'seagreen', 'mediumpurple']

for ax, (variant, res) in zip(axes, ab_results.items()):
    for idx, method in enumerate(['wald', 'wilson', 'agresti_coull', 'clopper_pearson']):
        lower, upper = res[method]
        ax.plot([lower, upper], [idx, idx], marker='o', linewidth=2, color=colors[idx], label=method_labels[idx])
    ax.axvline(res['p_hat'], color='black', linestyle='--', linewidth=2, label='p̂')
    ax.set_yticks(range(len(method_labels)))
    ax.set_yticklabels(method_labels)
    ax.set_xlabel('Click-through rate')
    ax.set_title(f"{variant}: 95% CI
(n={res['n']}, x={res['x']}, p̂={res['p_hat']:.4f})")
    ax.grid(True, axis='x', alpha=0.3)
    ax.legend(loc='lower right')

plt.tight_layout()
plt.show()

variant_keys = list(ab_results.keys())
p_a, p_b = ab_results[variant_keys[0]]['p_hat'], ab_results[variant_keys[1]]['p_hat']
n_a, n_b = ab_results[variant_keys[0]]['n'], ab_results[variant_keys[1]]['n']

diff = p_a - p_b
se_diff = np.sqrt(p_a * (1.0 - p_a) / n_a + p_b * (1.0 - p_b) / n_b)
z = stats.norm.ppf(0.975)
ci_diff = (diff - z * se_diff, diff + z * se_diff)
print(f"Difference in CTR ({variant_keys[0]} - {variant_keys[1]}): {diff:.4f}")
print(f"95% CI for difference: [{ci_diff[0]:.4f}, {ci_diff[1]:.4f}]")
if ci_diff[0] > 0:
    print('Conclusion: first variant significantly higher (α = 0.05)')
elif ci_diff[1] < 0:
    print('Conclusion: second variant significantly higher (α = 0.05)')
else:
    print('Conclusion: no significant difference (α = 0.05)')



### Exercise 4: Delta Method CI for the Coefficient of Variation

We approximate the standard error using the delta method and cross-check with a bootstrap.



In [ ]:

def cv_confidence_interval(sample, confidence=0.95):
    sample = np.asarray(sample, dtype=float)
    n = sample.size
    xbar = sample.mean()
    s = sample.std(ddof=1)
    cv_hat = s / xbar
    se_cv = cv_hat * np.sqrt((cv_hat**2 + 0.5) / n)
    alpha = 1.0 - confidence
    z = stats.norm.ppf(1.0 - alpha / 2.0)
    ci = (cv_hat - z * se_cv, cv_hat + z * se_cv)
    return cv_hat, se_cv, ci

height_sample = rng.normal(loc=170.0, scale=10.0, size=100)
cv_hat, se_cv, ci_delta = cv_confidence_interval(height_sample, confidence=0.95)

boot_draws = 5_000
bootstrap_cv = np.empty(boot_draws)
for b in range(boot_draws):
    bootstrap_sample = rng.choice(height_sample, size=height_sample.size, replace=True)
    bootstrap_cv[b] = bootstrap_sample.std(ddof=1) / bootstrap_sample.mean()

ci_boot = np.percentile(bootstrap_cv, [2.5, 97.5])

print('Coefficient of variation analysis')
print('=' * 40)
print(f'Sample size: n = {height_sample.size}')
print(f'Sample mean: {height_sample.mean():.2f}')
print(f'Sample sd: {height_sample.std(ddof=1):.2f}')
print(f'CV estimate: {cv_hat:.4f}')
print(f'Delta-method SE: {se_cv:.4f}')
print(f'Delta-method 95% CI: [{ci_delta[0]:.4f}, {ci_delta[1]:.4f}]')
print(f'Bootstrap 95% CI: [{ci_boot[0]:.4f}, {ci_boot[1]:.4f}]')

fig, axes = plt.subplots(1, 2, figsize=(13, 5))
axes[0].errorbar([0], [cv_hat], yerr=[[cv_hat - ci_delta[0]], [ci_delta[1] - cv_hat]], fmt='o', capsize=10, linewidth=2)
axes[0].set_xticks([0])
axes[0].set_xticklabels(['CV'])
axes[0].set_ylabel('Coefficient of variation')
axes[0].set_title('Delta-method interval')
axes[0].grid(True, axis='y', alpha=0.3)

axes[1].hist(bootstrap_cv, bins=40, density=True, alpha=0.7, edgecolor='black')
axes[1].axvline(cv_hat, color='red', linestyle='--', linewidth=2, label='Estimate')
axes[1].axvline(ci_boot[0], color='blue', linestyle='--', linewidth=2, label='Bootstrap CI')
axes[1].axvline(ci_boot[1], color='blue', linestyle='--', linewidth=2)
axes[1].set_xlabel('Coefficient of variation')
axes[1].set_ylabel('Density')
axes[1].set_title('Bootstrap distribution')
axes[1].legend()
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()


## 6. Summary and Key Takeaways

This notebook demonstrated:
1. Fisher information quantifies the information content of data
2. CRLB sets theoretical minimum variance for unbiased estimators
3. Sample mean achieves CRLB for Normal and Poisson models
4. MLE achieves CRLB asymptotically for exponential rate
5. Tradeoffs between bias and efficiency in finite samples

Key insights:
- Fisher information I(θ) = Var(U(θ)) measures precision
- CRLB = 1/I_n(θ) is the theoretical efficiency bound
- MLEs achieve CRLB asymptotically under regularity conditions
- Lesson 2 estimators are asymptotically efficient
- Finite-sample properties may differ from asymptotic behavior